In [ ]:
!pip install gensim nltk
!wget https://raw.githubusercontent.com/bakrianoo/aravec/master/utilities.py
# Download via terminal commands
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip"
!unzip "full_grams_cbow_100_twitter.zip"
!gdown 1pHt2cR21oQyeFydiYfF3lehKPm6qon5a
!gdown 1dgZvhDEp0xHvu0jCLNNW_s1mAfvnM8jU

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-01-06 18:32:26--  https://raw.githubusercontent.com/bakrianoo/aravec/master/utilities.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2412 (2.4K) [text/plain]
Saving to: ‘utilities.py’

utilities.py        100%[===================>]   2.36K  --.-KB/s    in 0s      

2023-01-06 18:32:26 (31.4 MB/s) - ‘utilities.py’ saved [2412/2412]

--2023-01-06 18:32:26--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connec

In [ ]:
import gensim
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils import to_categorical 
import collections
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
from nltk import ngrams
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import sys
import argparse
from utilities import * 

In [ ]:
model_t = gensim.models.Word2Vec.load("full_grams_cbow_100_twitter.mdl")
print("We've",len(model_t.wv.index2word),"vocabularies")

We've 1476715 vocabularies


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ♥️'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
stop_words = list(set(stopwords.words('arabic')))
print(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['لولا', 'جنيه', 'لكنَّ', 'لكيلا', 'دواليك', 'مساء', 'بعدا', 'بك', 'جيم', 'لك', 'يا', 'شتان', 'إياهما', 'من', 'نحو', 'إليكم', 'بعض', 'شبه', 'تفعلون', 'سبعين', 'أهلا', 'يونيو', 'وَيْ', 'هكذا', 'عاشر', 'نيسان', 'منذ', 'أكتوبر', 'ؤ', 'تلقاء', 'بؤسا', 'ّأيّان', 'إياه', 'ليسا', 'ج', 'أن', 'تعسا', 'شيكل', 'سبع', 'لات', 'ابتدأ', 'إياهن', 'ظاء', 'تسعمئة', 'جمعة', 'ءَ', 'لعلَّ', 'كلّما', 'مئتان', 'أوت', 'قاف', 'اثنان', 'خبَّر', 'ريث', 'هلّا', 'ذِه', 'بيد', 'اللتان', 'واو', 'ما', 'أمد', 'قد', 'ذال', 'ذواتا', 'بس', 'جلل', 'واحد', 'ذلكن', 'آها', 'ذاك', 'فمن', 'إياهم', 'غادر', 'تحوّل', 'اللتين', 'ستة', 'غ', 'فإذا', 'لئن', 'هاتي', 'كل', 'فيما', 'راح', 'ثمانية', 'فو', 'لكم', 'هاته', 'إياك', 'إياكما', 'أنّى', 'أيها', 'ثمانون', 'أربعمئة', 'إليك', 'قرش', 'خاء', 'خمسين', 'إي', 'لم', 'مثل', 'حين', 'هَجْ', 'هنا', 'ستين', 'لاسيما', 'فلا', 'خلف', 'باء', 'ذو', 'و', 'فيه', 'اثني', 'تسعين', 'أربعمائة', 'كأن', 'بهن', 'ثلاثون', 'تسعون', 'حرى', 'كما', 'كانون', 'كليكما', 'هؤلاء', 'ط', 'ثلاثة', 'يفعلون', 'تانِك', 'ظ

In [ ]:
train_neg = pd.read_csv('train_Arabic_tweets_negative_20190413.tsv', sep='\t',header=None)

In [ ]:
train_pos = pd.read_csv('train_Arabic_tweets_positive_20190413.tsv', sep='\t',header=None)

In [ ]:
def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, ' ', str(text))
    return text

In [ ]:
def remove_emoji(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, ' ', data)

In [ ]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    text = remove_emoji(text)
    text = remove_diacritics(text)
    text = re.sub(r'\s*[A-Za-z]+\b', ' ' , text)
    text = re.sub('[a-zA-Z0-9_]|#|http\S+', ' ', text)

    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text

In [ ]:
train = {}
for ind in train_neg.index:
  if len(train_neg[1][ind].split()) <185:
    train[train_neg[1][ind]] = 0
  
for ind in train_pos.index:
  if len(train_pos[1][ind].split()) <185:
    train[train_pos[1][ind]] = 1
  # print(train_neg[1][ind])

train_processed = {}
for item in train.items():
  train_processed[clean_text(item[0])]=item[1]
  

In [ ]:
train_data = pd.DataFrame(train_processed.items(), columns=['sentence', 'result'])
train_data = train_data.sample(frac=1).reset_index(drop=True)


In [ ]:
train_x = np.array(train_data['sentence'])
train_y = np.array(train_data['result'])

In [ ]:
model_t.wv[word_tokenize(train_x[0])[0]]

array([ 1.1363635 , -1.7163424 , -2.9076424 ,  3.356724  ,  0.957967  ,
       -0.79834336,  0.5437942 ,  0.87912893,  3.2863574 , -1.9490956 ,
        0.10482518,  0.5248643 , -1.8943369 , -2.1099353 ,  3.2023983 ,
       -2.2903132 ,  2.0430503 ,  0.47477594, -2.7887082 ,  2.6614218 ,
        0.74114764, -2.0767927 , -3.5537617 , -1.7487298 ,  3.3405063 ,
        0.56770444, -0.9474646 , -1.3589689 , -2.195783  ,  0.4432086 ,
       -0.23473713, -0.33584034,  1.2357498 ,  1.2607098 , -1.028466  ,
        1.6956838 ,  2.9493747 ,  1.8167673 ,  0.04696136, -0.80698305,
        0.3031963 ,  2.068561  ,  2.870488  , -0.164916  ,  0.2807077 ,
        2.632488  ,  1.6238861 ,  1.3463742 , -3.3742483 , -0.74782485,
       -1.5089936 , -2.0864596 , -3.6887248 , -0.73817927,  0.3436764 ,
        0.10736952, -3.3624833 ,  0.49278888, -1.2913638 , -1.5190442 ,
       -1.3642406 ,  1.1418374 ,  0.32636404,  1.9805261 , -0.37740672,
        0.83814704,  0.7860704 ,  2.7468817 ,  0.39045057, -1.27

In [ ]:
train_x[0]

'الساعه يوم الجمعة اجازة وراحة الرتويت الاجباري والتبادل ودي الاعضاء وتقبل الل…'

In [ ]:
train_x_input = np.zeros((len(train_x),184,100),dtype="float32")
print(train_x_input.shape)
for i in range(len(train_x)):
  index = 0
  for word in train_x[i].split():
    vec = np.zeros((100,),dtype="float32")
    if word in model_t.wv:
      vec+=model_t.wv[word]
    train_x_input[i,index]=vec
  index+=1
  

(28720, 184, 100)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100,input_shape=(184,100),),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
#               optimizer=tf.keras.optimizers.Adam(1e-4),


In [ ]:
# tf.config.run_functions_eagerly(True)

In [ ]:
history = model.fit(x=train_x_input,y=train_y,epochs=10,batch_size=8,verbose=1, validation_split=0.20)

Epoch 1/10
2872/2872 [==============================] - 106s 36ms/step - loss: 0.6930 - accuracy: 0.5071 - val_loss: 0.6937 - val_accuracy: 0.5035
Epoch 2/10
2872/2872 [==============================] - 106s 37ms/step - loss: 0.6930 - accuracy: 0.5097 - val_loss: 0.6938 - val_accuracy: 0.5035
Epoch 3/10
2872/2872 [==============================] - 105s 37ms/step - loss: 0.6929 - accuracy: 0.5112 - val_loss: 0.6931 - val_accuracy: 0.5035
Epoch 4/10
2872/2872 [==============================] - 105s 36ms/step - loss: 0.6930 - accuracy: 0.5116 - val_loss: 0.6932 - val_accuracy: 0.5035
Epoch 5/10
2872/2872 [==============================] - 104s 36ms/step - loss: 0.6930 - accuracy: 0.5113 - val_loss: 0.6938 - val_accuracy: 0.5035
Epoch 6/10
2872/2872 [==============================] - 104s 36ms/step - loss: 0.6930 - accuracy: 0.5119 - val_loss: 0.6931 - val_accuracy: 0.5035
Epoch 7/10
2872/2872 [==============================] - 105s 37ms/step - loss: 0.6930 - accuracy: 0.5119 - val_loss: 0

In [ ]:
!gdown 1x2bNmaWCI-ji1-JLSI6N22tIINVv5yhD
!gdown 1L6pLW10tTO1ZM7JZU4MroPABZxAPKPh7

Downloading...
From: https://drive.google.com/uc?id=1x2bNmaWCI-ji1-JLSI6N22tIINVv5yhD
To: /content/test_Arabic_tweets_negative_20190413.tsv
100% 696k/696k [00:00<00:00, 95.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1L6pLW10tTO1ZM7JZU4MroPABZxAPKPh7
To: /content/test_Arabic_tweets_positive_20190413.tsv
100% 776k/776k [00:00<00:00, 103MB/s]


In [ ]:
test_neg = pd.read_csv('test_Arabic_tweets_negative_20190413.tsv', sep='\t',header=None)

In [ ]:
test_pos = pd.read_csv('test_Arabic_tweets_positive_20190413.tsv', sep='\t',header=None)

In [ ]:
test = {}
for ind in test_neg.index:
  if len(test_neg[1][ind].split()) <185:
    test[test_neg[1][ind]] = 0
  
for ind in test_pos.index:
  if len(test_pos[1][ind].split()) <185:
    test[test_pos[1][ind]] = 1
  # print(train_neg[1][ind])

In [ ]:
test_processed = {}
for item in test.items():
  test_processed[clean_text(item[0])]=item[1]
  

In [ ]:
test_data = pd.DataFrame(test_processed.items(), columns=['sentence', 'result'])


In [ ]:
test_x = test_data['sentence']
test_y = test_data['result']

In [ ]:
test_x_input = np.zeros((len(test_x.values),184,100),dtype="float32")
print(test_x_input.shape)
for i in range(len(test_x.values)):
  index = 0
  for word in test_x[i].split():
    vec = np.zeros((100,),dtype="float32")
    if word in model_t.wv:
      vec+=model_t.wv[word]
    test_x_input[i,index]=vec
  index+=1
  

(8672, 184, 100)


In [ ]:
score = model.evaluate(test_x_input, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6934095025062561
Test accuracy: 0.504497230052948
